<a href="https://colab.research.google.com/github/tcapelle/hackercup_rag/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{rag-hackercup} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />


# Introduction


</a>


In this notebook, we will build a few Code Generation agents for the [HackerCup AI](https://hackercupai.github.io/) challenge.

We will build three different agents using different techniques and evaluate them using [W&B Weave](https://weave-docs.wandb.ai/).


<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/static/img/evals-hero.png" width="800" height="450">

A more detailed walkthough of the approach we will use in this notebook can be found in the following Youtube video:
Hint: Click on the image to watch the video 😎

<a target="_blank" href="https://www.youtube.com/watch?v=cObBj2UpWK8">
<img src="https://img.youtube.com/vi/cObBj2UpWK8/0.jpg" width="600" height="450">
</a>

## Weave


Weave is a lightweight toolkit for tracking and evaluating LLM applications, built by Weights & Biases. We will use the following weave to trace and evaluate the various agents we build.

We will use Weave to keep track and evaluate the different agents we build.

Our goal is to bring rigor, best-practices, and composability to the inherently experimental process of developing AI applications, without introducing cognitive overhead.

If you want to learn more about Weave, you can [get started](https://weave-docs.wandb.ai/quickstart) by decorating Python functions with `@weave.op`.

## Setup 

**Note: You need to run this cell only once**
We will clone the starter-kits repo
Set the rag folder as our working directory
and install the dependencies for the project.

**You can comment out the cell after you have run it once.**

In [ ]:
# Clone the starter-kits repo
!git clone https://github.com/tcapelle/hackercup_rag
# Change directory to the rag folder. Running the next line twice in the same session will raise an error.
%cd hackercup_rag
# Install dependencies
!pip install -r requirements.txt

In [ ]:
import weave

WEAVE_PROJECT = "hackercup"
weave_client = weave.init(WEAVE_PROJECT)

## Dataset
We will use [HackerCup dataset](https://huggingface.co/datasets/hackercupai/hackercup) in this notebook.

Specifically, the **practice** dataset from the **2023** season.

We have already processed the dataset and saved it as a [`weave.Dataset`](https://weave-docs.wandb.ai/guides/core-types/datasets/). You can either use the Dataset by running the next cell or download the dataset using the instructions below.

We will use the dataset to load some practice problems and solutions from the HackerCup dataset and evaluate our agents on it.

In [ ]:
import os

os.environ["BASE_URL"] = "http://195.242.24.252:8000/v1"
os.environ["MAX_TOKENS"] = "2048"
os.environ["WEAVE_PARALLELISM"] = "1"

from utils import (
    Problem,
    FAST_LLM,
    STRONG_LLM,
)


practice_dataset_uri = "weave:///parambharat/hackercup/object/practice_dataset:R35fXf9N3FE2IOesg7bRPaPAxiE9YbpirhXO9HcHs8w"
problems_dataset = weave.ref(practice_dataset_uri).get().rows[:]
problems = list(map(lambda x: Problem(**x), problems_dataset))
problem = problems[0]
print("Sample Problem:\n\n", problem.model_dump_json(indent=2))

Alternatively, you can download the dataset by running the download script from the [submit-first-solution](https://github.com/HackerCupAI/starter-kits/tree/main/submit_first_solution). Specifically, you can run the following command to download the dataset:

```bash
python download.py --year 2023 --dataset_folder data
```


This should create a `dataset` folder with the problems and solutions.

Here's an example of what the data looks like for the `dim_sum_delivery` problem from the `2023` season:

```
data/dataset/2023/practice
...
├── dim_sum_delivery.cpp
├── dim_sum_delivery.in
├── dim_sum_delivery.md
├── dim_sum_delivery.out
├── dim_sum_delivery_sample_input.txt
├── dim_sum_delivery_sample_output.txt
├── dim_sum_delivery_sol.md
...
```

Each problem has a `in`, `out`, `md`, `cpp`, and `sol` file.

The `in` file contains the input data for the problem.
The `out` file contains the expected output for the problem.
The `md` file contains the problem statement.
The `cpp` file contains the source code to the solution.
The `sol` file contains the detailed solution to the problem.
The `sample_input.txt` and `sample_output.txt` files contain the sample input and output for the problem. These are the test cases that will be available to the agent during development and evaluation.

In [ ]:
import asyncio
import logging

from nest_asyncio import apply

apply()

# Some logging to see the progress
logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

logger = logging.getLogger(__name__)

Now that we have a way to load a problem and evaluate a solution, let's define a prompt to solve the problem and create a simple agent to solve the problem. 

Here'e one such prompt we will use to solve the problem, it contains instructions for the model on how to solve the problem and the format of the response we expect from the model. Feel free to tweak the prompt if you like but this should work decently well for our use case.

## Reflection Agent



While the RAG agent is an improvement over the zero-shot agent, it's still not perfect.
It's still susceptible to hallucinations and incorrect solutions. 
One way to mitigate this is to use reflection.
We can use another LLM call to reflect on the solution and test results and improve it.
We can then use the improved solution to generate new few-shot examples and repeat the process in a loop until we converge to a solution or the iteration limit is reached.

Again, this is not the best approach to solve the problem and has a lot of room for improvement, but it should help us get towards a working solution.

Here are the reflection instructions we will provide to the LLM to reflect on the solution and test results, feel free to change the instructions to improve the agent's performance.

In [ ]:
from agent import rag_solver
from retriever import Retriever

retriever = Retriever()

from agent import REFLECTION_INSTRUCTIONS, rework_solution

print(REFLECTION_INSTRUCTIONS)

In [ ]:
@weave.op
async def rag_solver_with_reflection(
        retriever: Retriever,
        problem: Problem,
        model: str = FAST_LLM,
        temperature: float = 0.1,
        max_iterations: int = 2,
        timeout: int = 10,
):
    num_iterations = 0
    test_report = "failed"
    solution = None
    while not test_report == "passed" and num_iterations < max_iterations:
        rag_result = await rag_solver(
            retriever=retriever,
            problem=problem,
            timeout=timeout,
            model=model,
            temperature=temperature,
        )
        solution = rag_result["solution"]
        test_report = rag_result["test_report"]
        if test_report == "passed":
            return rag_result
        rework_result = await rework_solution(
            problem=problem,
            incorrect_solution=solution,
            test_report=test_report,
            model=model,
            temperature=temperature,
            timeout=timeout,
        )
        solution = rework_result["solution"]
        test_report = rework_result["test_report"]
        if test_report == "passed":
            return {
                "solution": solution,
                "stage": "reflection",
                "test_report": test_report,
            }
        num_iterations += 1
    logger.info("Failed to generate a solution")
    return {"solution": solution, "stage": "failed", "test_report": test_report}

In [ ]:
reflection_result = await rag_solver_with_reflection(
    retriever, problem, max_iterations=2, timeout=30
)

print("*" * 80)
print(reflection_result["solution"].source_code)
print("*" * 80)
print(reflection_result["test_report"])

Great, now, we are ready to evaluate a more complex agent that uses reflection
This agent will try to solve the problem using the retriever
and if it fails, it will ask the model to reflect on the problem
and then re-work the solution
and repeat this process for a fixed number of iterations
or until the solution is correct or the iteration limit is reached

But the best part is that we can use the same evaluation framework we used for the zero-shot and RAG agent to evaluate the RAG reflection agent.

In [ ]:
class RAGReflectionAgent(weave.Model):
    retriever: Retriever
    max_iterations: int = 2
    timeout: int = 30
    model: str = STRONG_LLM
    temperature: float = 0.0

    @weave.op
    async def predict(self, problem: Problem):
        return await rag_solver_with_reflection(
            self.retriever,
            Problem(**problem),
            model=self.model,
            temperature=self.temperature,
            max_iterations=self.max_iterations,
            timeout=self.timeout,
        )

In [ ]:
# This is a simple depection of the evaluation.
# We expect the output to be `"passed"` for all the problems if the agent is working correctly.
examples = [{"problem": problem, "expected": "passed"} for problem in problems]


# A simple scorer that checks if the code generated by agent passed the test case
@weave.op
def scorer(expected: str, model_output: dict) -> dict:
    return {"passed": expected == model_output["test_report"]}


# This is a simple evaluation that checks if the code generated by agent passed the test
evaluation = weave.Evaluation(dataset=examples, scorers=[scorer])

In [ ]:
# Evaluate the RAG reflection agent for all the models and temperatures
tasks = []

LLM = STRONG_LLM
eval_temperatures = [0.0, 0.5]


for temperature in eval_temperatures:
        rag_reflection_agent = RAGReflectionAgent(
            retriever=retriever, model=LLM, temperature=temperature, timeout=30
        )
        rag_reflection_results = evaluation.evaluate(rag_reflection_agent)
        tasks.append(rag_reflection_results)
rag_reflection_results = await asyncio.gather(*tasks)

Okay, that completes the demo!

Key takeaways from this demo:
1. We tried to solve some challenging competitive programming problems using LLM agents.
2. We tried three different agents:
    - Zero-shot agent
    - RAG agent
    - RAG reflection agent
3. We used Weave to evaluate the agents and compare their performance.

We hope you found this demo useful and interesting and that it gave you some ideas on how to use LLM agents to solve challenging problems.
